# Lesson 3: Vectorstores and embeddings

# Vectorstore ingestion

In [1]:
// Import environment variables from .env file for secure API access

import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
// Import the OpenAIEmbeddings class for generating text embeddings
import { OpenAIEmbeddings } from "@langchain/openai";

// Create an instance of OpenAIEmbeddings
const embeddings = new OpenAIEmbeddings();

// Generate an embedding for the provided text and await the result
await embeddings.embedQuery("This is some sample text");

[
   -0.010393001,   0.0024292984, -0.00073567877,  -0.010911339,    -0.01144936,
    0.022924965,   -0.014644682,   0.0017452893,  -0.017505383,   -0.019303162,
    0.005166975,    0.034170926,   -0.012210463,  0.0019372054,    0.004691285,
    0.013155282,    0.024591519,   0.0018027001,   0.004596147,  -0.0062331758,
  -0.0051505715, -0.00069098035,   -0.008208108,   0.013975437,   -0.008962651,
  -0.0040417225,  -0.0007192757,   -0.019657468,  0.0040154774,  -0.0017288862,
    0.016048787,   -0.021862045, -0.00078570825,  -0.022295086,   0.0062561403,
    0.007040208,   -0.011259085,   -0.013417731,   0.028869446,   -0.023567967,
   0.0068892995,     0.01743977,    0.017308546, -0.0059444816,  -0.0038448854,
   -0.014369111,  -0.0017502102,  -0.0059444816,  -0.007912853,    0.010458613,
    0.032806188, -0.00083901826,   -0.023987886, -0.0033823182, -0.00079801056,
   0.0025769263,   -0.022400066,    0.025168909,   0.019775571,    0.010773553,
   -0.011705249,   -0.002529357,   0.0

In [6]:
// Import the similarity function from the 'ml-distance' package.
// This function is typically used to calculate similarity or distance between vectors.
import { similarity } from "ml-distance";

// Assuming an 'embeddings' object from previous context, which is an instance of some model capable of generating text embeddings.

// Generate an embedding for a query about vectors in machine learning.
// 'embedQuery' is an asynchronous function that returns the embedding of the input text.
const vector1 = await embeddings.embedQuery(
    "What are vectors useful for in machine learning?"
);

// Generate an embedding for a query about parrots, likely unrelated to the first query.
// This demonstrates generating embeddings for vastly different pieces of text to compare their similarity later.
const unrelatedVector = await embeddings.embedQuery(
    "A group of parrots is called a pandemonium."
);

In [4]:
similarity.cosine(vector1, unrelatedVector);

0.6961084034193643

In [7]:
// Generate an embedding for a query about vector representations in information.
const similarVector = await embeddings.embedQuery(
    "Vectors are representations of information."
);

// Calculate the cosine similarity between 'vector1' and 'similarVector'.
// This measures how similar the two text embeddings are, with values closer to 1 indicating high similarity.
similarity.cosine(vector1, similarVector);


0.8588060436065783

In [8]:
// Importing the PDF parsing utility.
import * as parse from "pdf-parse";

// Import PDFLoader for loading PDF files from the filesystem.
import { PDFLoader } from "langchain/document_loaders/fs/pdf";

// Import RecursiveCharacterTextSplitter for splitting text into smaller chunks.
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

// Initialize PDFLoader with the path to the PDF document.
const loader = new PDFLoader("./data/MachineLearning-Lecture01.pdf");

// Load the PDF document using the loader.
const rawCS229Docs = await loader.load();

// Initialize the text splitter with specified chunk size and overlap.
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 128,  // Size of each text chunk.
  chunkOverlap: 0, // No overlap between chunks.
});

// Split the loaded PDF document into chunks using the text splitter.
const splitDocs = await splitter.splitDocuments(rawCS229Docs);


In [9]:
// Import MemoryVectorStore for storing vectors in memory.
import { MemoryVectorStore } from "langchain/vectorstores/memory";

// Initialize a MemoryVectorStore with preloaded embeddings.
const vectorstore = new MemoryVectorStore(embeddings);

In [10]:
// Add the split documents to the in-memory vector store.
await vectorstore.addDocuments(splitDocs);


In [11]:
// Perform a similarity search in the vector store using a query.
const retrievedDocs = await vectorstore.similaritySearch(
    "What is deep learning?", // The search query.
    4                          // Number of similar documents to retrieve.
);

// Extract the page content from each retrieved document.
const pageContents = retrievedDocs.map(doc => doc.pageContent);

// The variable 'pageContents' now holds the content of the pages
// most similar to the search query.
pageContents


[
  "piece of research in machine learning, okay?",
  "are using a learning algorithm, perhaps without even being aware of it.",
  "some of my own excitement about machine learning to you.",
  "of the class, and then we'll start to talk a bit about machine learning."
]

# Retrievers

In [12]:
// Convert the vector store into a retriever for document retrieval.
const retriever = vectorstore.asRetriever();

In [13]:
// same documents about deep learning
await retriever.invoke("What is deep learning?")

[
  Document {
    pageContent: "piece of research in machine learning, okay?",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 8, lines: { from: 2, to: 2 } }
    }
  },
  Document {
    pageContent: "are using a learning algorithm, perhaps without even being aware of it.",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",